# 使用TensorFlow实现卷积神经网络LENET

用于MNist数据库识别的卷积神经网络Lenet的定义如下：
![CNN](http://personal.ie.cuhk.edu.hk/~ccloy/project_target_code/images/fig3.png)


In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from tensorflow.examples.tutorials.mnist import input_data

import tensorflow as tf
import numpy as np

mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
learning_rate = 0.001
batch_size = 128
num_step = 500
display_step = 10

num_input = 784
num_classes = 10
dropout = 0.75

X = tf.placeholder(dtype=tf.float32, shape=[None, num_input])
Y = tf.placeholder(dtype=tf.float32, shape=[None, num_classes])
keep_prob = tf.placeholder(dtype=tf.float32)

In [ ]:
# 定义一些函数

def conv2d(x, W, b, stride= 1):
    x = tf.nn.conv2d(x, W,strides=[1, stride, stride, 1], padding="SAME")
    x = tf.nn.bias_add(x , b)
    return tf.nn.relu(x)

def maxpooling2d(x, k = 2):
    return tf.nn.max_pool(x, ksize=[1,k,k,1], strides=[1,k,k,1], padding="SAME")

def conv_net(x,weights, biases,dropout):
    x = tf.reshape(x, shape=[-1, 28,28,-1])
    
    